In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
import warnings
warnings.filterwarnings('ignore')

In [2]:
from rpy2 import robjects
from rpy2.robjects import Formula

from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.packages import importr

base = importr("base")
stats = importr("stats")
DESeq2 = importr("DESeq2")

# Все 35 трехпетельных белков человека 

In [3]:
upar = pd.read_csv('human_upar.tsv', index_col=0, sep='\t')
upar = set(upar['Approved symbol'])
print(len(upar))

35


# Функция подсчета дифференциальной экспрессии 

In [4]:
def deseq(meta, counts, formula, ref):
    meta["Tissue"] = stats.relevel(robjects.vectors.FactorVector(meta["Tissue"]), ref=ref)

    # Calculate normalization factors
    dds = DESeq2.DESeqDataSetFromMatrix(countData=counts, colData=meta, design=Formula(formula))
    dds = DESeq2.DESeq(dds)

    res = DESeq2.results(dds, name=f"Tissue_AD_vs_{ref}")
    res = DESeq2.lfcShrink(dds, coef=f"Tissue_AD_vs_{ref}", type="apeglm")
    res = pd.DataFrame(base.as_data_frame(res))
    res.index = counts.index
    res = res.sort_values("padj")
    res = res.loc[res["padj"] < 0.05]
    res = res.loc[res["log2FoldChange"].abs() > 0.5]

    return res

# Проект GSE159699 

In [5]:
counts = pd.read_csv('GSE159699/GSE159699_summary_count.star.txt', index_col=0, sep='\t').reindex(upar).dropna()
# counts = counts[counts.T.median() > 5]
print(counts.shape)
counts

(33, 30)


,21-1A-AD,20-1T-AD,23-2A-AD,22-2T-AD,26-3A-AD,24-3T-AD,27-5A-AD,25-5T-AD,29-6T-AD,31-7T-AD,...,10-8A-Old,17-9A-Old,2-12A-Young,4-13A-Young,6-14A-Young,8-15A-Young,9-16A-Young,3-17T-Young,5-18T-Young,7-19T-Young
refGene,,,,,,,,,,,,,,,,,,,,,
LY6G6D,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,3.0,...,0.0,0.0,3.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0
LYPD6B,10.0,23.0,12.0,0.0,7.0,1.0,5.0,4.0,17.0,9.0,...,29.0,12.0,26.0,22.0,18.0,0.0,16.0,2.0,2.0,5.0
LY6G5B,34.0,54.0,47.0,75.0,42.0,64.0,31.0,67.0,43.0,32.0,...,61.0,42.0,37.0,27.0,33.0,39.0,67.0,55.0,72.0,56.0
LY6G5C,27.0,16.0,27.0,9.0,18.0,27.0,16.0,16.0,18.0,24.0,...,5.0,9.0,14.0,12.0,5.0,6.0,10.0,6.0,19.0,12.0
PLAUR,82.0,53.0,82.0,98.0,74.0,48.0,47.0,87.0,41.0,80.0,...,31.0,81.0,69.0,80.0,26.0,95.0,60.0,75.0,80.0,62.0
GPIHBP1,70.0,236.0,107.0,1168.0,316.0,176.0,30.0,1537.0,602.0,33.0,...,114.0,226.0,461.0,97.0,57.0,2141.0,46.0,3622.0,2085.0,933.0
LY6G6C,4.0,1.0,3.0,1.0,2.0,3.0,5.0,1.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
SLURP1,6.0,1.0,2.0,5.0,4.0,10.0,8.0,1.0,4.0,14.0,...,0.0,4.0,2.0,0.0,1.0,0.0,2.0,1.0,2.0,4.0
GML,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


for i, j in [(0, 12), (12, 22), (22, 30)]:
    print(counts.iloc[:, i:j].loc['PSCA'].median())

In [6]:
print('Генов белков, которых нет в проекте', set(upar) - set(counts.index))

Генов белков, которых нет в проекте {'LY6L', 'SLURP2'}


In [7]:
counts.columns.str.split('-').str[-1].value_counts()

AD       12
Old      10
Young     8
dtype: int64

In [8]:
old_df = counts.iloc[:, :22]
meta = pd.DataFrame({"Tissue": ['AD']*12 + ['Old']*10}, index=old_df.columns)
old = deseq(meta, old_df, "~ Tissue", 'Old')
old.columns = 'AD_Old_' + old.columns
old = old[['AD_Old_log2FoldChange', 'AD_Old_padj']]
old

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,AD_Old_log2FoldChange,AD_Old_padj
refGene,,
LYNX1,-0.627832,0.000355
PINLYP,0.729596,0.008559


In [9]:
young_df = pd.concat([counts.iloc[:, :12], counts.iloc[:, -8:]], axis=1)
meta = pd.DataFrame({"Tissue": ['AD']*12 + ['Young']*8}, index=young_df.columns)
young = deseq(meta, young_df, "~ Tissue", 'Young')
young.columns = 'AD_Young_' + young.columns
young = young[['AD_Young_log2FoldChange', 'AD_Young_padj']]
young

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,AD_Young_log2FoldChange,AD_Young_padj
refGene,,
LYNX1,-0.513935,0.042246
LY6G6F,1.481236,0.047474
SLURP1,1.136846,0.047474
GPIHBP1,-1.145622,0.049386
LY6G6C,1.191178,0.049386


In [10]:
alls_df = counts.copy()
meta = pd.DataFrame({"Tissue": ['AD']*12 + ['Normal']*18}, index=alls_df.columns)
alls = deseq(meta, alls_df, "~ Tissue", 'Normal')
alls.columns = 'AD_Young_Old_' + alls.columns
alls = alls[['AD_Young_Old_log2FoldChange', 'AD_Young_Old_padj']]
alls

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: -- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

R[write to console]: estimating dispersions

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,AD_Young_Old_log2FoldChange,AD_Young_Old_padj
refGene,,
LYNX1,-0.579286,0.000472
PINLYP,0.651884,0.002305


In [11]:
control_df = counts.iloc[:, 12:]
meta = pd.DataFrame({"Tissue": ['AD']*10 + ['control']*8}, index=control_df.columns)
control = deseq(meta, control_df, "~ Tissue", 'control')
control.columns = 'Young_Old_' + control.columns
control = control[['Young_Old_log2FoldChange', 'Young_Old_padj']]
control

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,Young_Old_log2FoldChange,Young_Old_padj
refGene,,
GPIHBP1,-3.264609,0.000158


In [12]:
counts.shape, len(counts)

((33, 30), 33)

In [13]:
result = alls.join([old, young, control], how='outer')
# result.to_csv('results/GSE159699_result.tsv', sep='\t')
result

,AD_Young_Old_log2FoldChange,AD_Young_Old_padj,AD_Old_log2FoldChange,AD_Old_padj,AD_Young_log2FoldChange,AD_Young_padj,Young_Old_log2FoldChange,Young_Old_padj
LYNX1,-0.579286,0.000472,-0.627832,0.000355,-0.513935,0.042246,NaN,NaN
PINLYP,0.651884,0.002305,0.729596,0.008559,NaN,NaN,NaN,NaN
LY6G6F,NaN,NaN,NaN,NaN,1.481236,0.047474,NaN,NaN
SLURP1,NaN,NaN,NaN,NaN,1.136846,0.047474,NaN,NaN
GPIHBP1,NaN,NaN,NaN,NaN,-1.145622,0.049386,-3.264609,0.000158
LY6G6C,NaN,NaN,NaN,NaN,1.191178,0.049386,NaN,NaN


# Проект http://aging.brain-map.org 

In [14]:
# Загрузка дополнительных данных для разделения пациентов на группы
descr = pd.read_csv('atlas/DonorInformation.csv', index_col=0)
donor_id = pd.read_csv('atlas/columns-samples.csv', index_col=0)
genes = pd.read_csv('atlas/rows-genes.csv', index_col=0).gene_symbol.to_dict()

col = 'dsm_iv_clinical_diagnosis'
descr = descr[descr[col].isin(['No Dementia', "Alzheimer's Disease Type"])][col].to_dict()
donor_id['info'] = donor_id.donor_id.apply(descr.get)
donor_id = (donor_id['info'] + ' ' + donor_id.structure_acronym).to_dict()

In [15]:
fpkm = pd.read_csv('atlas/fpkm_table_normalized.csv', index_col=0)
fpkm.index = fpkm.index.to_series().apply(genes.get)
fpkm.columns = fpkm.columns.to_series().apply(lambda x: donor_id.get(int(x)))
list_col = fpkm.columns.value_counts().index.tolist()
fpkm = fpkm.reindex(upar).dropna()[list_col]
# fpkm = fpkm[fpkm.T.median() > 0]
print(fpkm.shape)
fpkm

(33, 303)


,No Dementia TCx,No Dementia TCx,No Dementia TCx,No Dementia TCx,No Dementia TCx,No Dementia TCx,No Dementia TCx,No Dementia TCx,No Dementia TCx,No Dementia TCx,...,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP,Alzheimer's Disease Type HIP
gene_id \ rnaseq_profile_id,,,,,,,,,,,,,,,,,,,,,
LY6G6D,0.000000,0.000000,0.000000,0.000000,0.187253,0.000000,0.000000,0.000000,0.204145,0.000000,...,0.177076,0.000000,0.266843,0.000000,0.000000,0.441102,0.000000,0.000000,0.000000,0.000000
LYPD6B,0.462984,0.873849,0.346477,0.830540,0.972990,0.606827,1.866495,0.999981,0.782790,0.117639,...,0.796957,2.276889,4.224096,1.148186,2.312956,2.895442,1.609846,2.989925,2.210371,2.838357
LY6G5B,2.663793,2.183498,2.142098,3.279771,2.257528,6.067300,1.974146,2.386403,1.399098,3.525474,...,1.188258,2.920764,1.178813,2.410148,2.657672,2.054110,2.508792,1.284646,2.034928,1.317973
LY6G5C,1.271733,0.591387,1.061100,0.691925,0.696941,0.000000,0.654448,0.893566,1.291531,0.576002,...,1.031010,0.828012,0.811847,0.799819,0.263593,0.528991,0.758871,0.381062,0.920234,0.257383
PLAUR,2.121441,3.427030,2.646145,1.745287,2.341226,2.912498,1.349863,1.313472,1.368288,2.049620,...,1.920127,1.917031,0.602146,1.247676,4.358949,4.019822,1.026049,1.982242,1.727840,2.985527
GPIHBP1,3.118672,2.876741,0.721234,2.772287,2.813043,2.455458,1.819824,2.048639,1.890681,4.190725,...,3.734607,4.823323,0.475271,5.681828,6.935389,3.933938,3.418516,2.740192,2.649448,0.690122
LY6G6C,0.000000,0.000000,0.000000,0.116541,0.000000,0.093772,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080268,0.073390
SLURP1,0.084712,0.000000,0.000000,0.191911,0.000000,0.179971,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.240284,0.108590,0.118755,0.000000,0.000000,0.000000,0.000000,0.000000
GML,0.134888,0.089253,0.122119,0.022179,0.184233,0.000000,0.000000,0.000000,0.120116,0.000000,...,0.095330,0.043439,0.543898,0.204139,0.049359,0.000000,0.000000,0.376537,0.000000,0.222929


In [ ]:
for name in fpkm.columns.str.split().str[-1].unique():
    for i in fpkm.index:
#         print(i, name)
        data = fpkm.loc[i][[f"Alzheimer's Disease Type {name}", f"No Dementia {name}"]].reset_index()
        sns.histplot(data=data, x=i, #alpha=0.6,
                     hue='index', bins=10, stat='density',
                     kde=True)
        plt.tight_layout()
        plt.savefig(f'figures/{i}_{name}.png', facecolor='white')
        plt.close()

In [ ]:
print('Генов белков, которых нет в проекте', set(upar) - set(fpkm.index))

In [ ]:
# Группы пациентов по болезни и области мозга
pval=0.06
inds = fpkm.columns.value_counts()
inds

In [ ]:
descr = pd.read_csv('atlas/DonorInformation.csv', index_col=0)
descr.head()

In [ ]:
pd.read_csv('atlas/columns-samples.csv', index_col=0)

# t-test 

In [ ]:
df = fpkm[['No Dementia HIP', "Alzheimer's Disease Type HIP"]]
n = 50
ttest = pd.DataFrame([ttest_ind(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index], index=df.index, columns=["t-test", 'p-value'])
ttest[ttest['p-value'] < pval].sort_values('p-value')

In [ ]:
df = fpkm[['No Dementia TCx', "Alzheimer's Disease Type TCx"]]
n = 50
ttest = pd.DataFrame([ttest_ind(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index], index=df.index, columns=["t-test", 'p-value'])
ttest[ttest['p-value'] < pval].sort_values('p-value')

In [ ]:
df = fpkm[['No Dementia FWM', "Alzheimer's Disease Type FWM"]]
n = 47
ttest = pd.DataFrame([ttest_ind(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index], index=df.index, columns=["t-test", 'p-value'])
ttest[ttest['p-value'] < pval].sort_values('p-value')

In [ ]:
df = fpkm[['No Dementia PCx', "Alzheimer's Disease Type PCx"]]
n = 46
ttest = pd.DataFrame([ttest_ind(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index],
                     index=df.index, columns=["t-test", 'p-value'])
ttest[ttest['p-value'] < pval].sort_values('p-value')

# mannwhitneyu 

In [ ]:
df = fpkm[['No Dementia HIP', "Alzheimer's Disease Type HIP"]]
n = 50
mw = pd.DataFrame([mannwhitneyu(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index],
                  index=df.index, columns=["mw_hip", 'p-value_hip'])
hip = mw[mw['p-value_hip'] < pval].sort_values('p-value_hip')
hip

In [ ]:
df = fpkm[['No Dementia TCx', "Alzheimer's Disease Type TCx"]]
n = 50
mw = pd.DataFrame([mannwhitneyu(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index],
                  index=df.index, columns=["mw_tcx", 'p-value_tcx'])
tcx = mw[mw['p-value_tcx'] < pval].sort_values('p-value_tcx')
tcx

In [ ]:
df = fpkm[['No Dementia FWM', "Alzheimer's Disease Type FWM"]]
n = 47
mw = pd.DataFrame([mannwhitneyu(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index], index=df.index,
                  columns=["mw_fwm", 'p-value_fwm'])
fwm = mw[mw['p-value_fwm'] < pval].sort_values('p-value_fwm')
fwm

In [ ]:
df = fpkm[['No Dementia PCx', "Alzheimer's Disease Type PCx"]]
n = 46
mw = pd.DataFrame([mannwhitneyu(df.loc[gene].iloc[:n], df.loc[gene].iloc[n:]) for gene in df.index],
                  index=df.index, columns=["mw_pcx", 'p-value_pcx'])
pcx = mw[mw['p-value_pcx'] < pval].sort_values('p-value_pcx')
pcx

In [ ]:
res = hip.join([fwm, tcx, pcx], how='outer')
print(res.shape)
# res.to_csv('results/atlas_result.tsv', sep='\t')
cols = ['mw_hip', 'mw_fwm', 'mw_tcx', 'mw_pcx',
        'AD_Young_Old_log2FoldChange', 'AD_Young_Old_padj',
       'AD_Old_log2FoldChange', 'AD_Young_log2FoldChange',
       'Young_Old_log2FoldChange',
       'Young_Old_padj']
# res = res.join(result, how='inner').drop(cols, axis=1)
res

In [ ]:
result

In [ ]:
ls = hip.join([fwm, tcx, pcx], how='outer').join(result, how='outer')
ls = ls[~ls.index.isin(result.index)]
ls

In [ ]:
fpkm.loc[(fpkm.T.median() > 0)]

In [ ]:
for j in res.index:
    tmp = []
    for i in fpkm.columns.unique():

        tmp.append(fpkm.loc[j, i].median())
    if sum(tmp) < 10:
        print(j, max(tmp))

In [ ]:
data = fpkm.copy()
inds = []
for j in res.index:
    tmp = []
    for i in fpkm.columns.unique():
        tmp.append(fpkm.loc[j, i].median())
    if min(tmp) > 1:
        inds.append(j)
data = data.loc[inds].sort_index()
print(data.shape)
fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(data=data, ax=ax, cmap='Blues')

inds = fpkm.columns.shape[0] / 8
xlab = list({i.get_text(): 0 for i in ax.get_xticklabels()}.keys())
ax.set_xticks([i * inds - (inds/2) for i in range(1, 9)])
ax.set_xticklabels(xlab, rotation=50, size=15)

ax.set_yticklabels(ax.get_yticklabels(), rotation=0, size=15)
ax.set_xlabel('Группы образцов', size=15)
ax.set_ylabel('Гены трехпетельных белков', size=15)
ax.vlines([i * inds for i in range(1, 8)], *ax.get_ylim())
plt.tight_layout()
plt.savefig('results/heatmap_medexpr_1', facecolor='white', format='png')
plt.show()

In [ ]:
data = fpkm.copy().loc[res.index].sort_index()

print(data.shape)
fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(data=data, ax=ax, cmap='Blues')

inds = fpkm.columns.shape[0] / 8
xlab = list({i.get_text(): 0 for i in ax.get_xticklabels()}.keys())
ax.set_xticks([i * inds - (inds/2) for i in range(1, 9)])
ax.set_xticklabels(xlab, rotation=50, size=15)

ax.set_yticklabels(ax.get_yticklabels(), rotation=0, size=15)
ax.set_xlabel('Группы образцов', size=15)
ax.set_ylabel('Гены трехпетельных белков', size=15)
ax.vlines([i * inds for i in range(1, 8)], *ax.get_ylim())
plt.tight_layout()
plt.savefig('results/heatmap_all_atlas', facecolor='white', format='png')
plt.show()

In [ ]:
lens = pd.read_csv('gencode.v34.lengths.tsv', index_col=0, sep='\t')['Length'].to_dict()

In [ ]:
data = pd.read_csv('GSE159699/GSE159699_summary_count.star.txt', index_col=0, sep='\t')
data = data / data.sum()
data = data.T.apply(lambda x: x * 1000000000 / lens.get(x.name, -1)).T
data = data.reindex(result.index)

print(data.shape)

fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(data=data, ax=ax, cmap='Blues')

inds = data.columns.shape[0] / 3
xlab = ["Alzheimer's disease", 'Old control', "Young control"]
ax.set_xticks([i * inds - (inds / 2) for i in range(1, 4)])
ax.set_xticklabels(xlab, rotation=50, size=15)

ax.set_yticklabels(ax.get_yticklabels(), rotation=0, size=15)
ax.set_xlabel('Группы образцов', size=15)
ax.set_ylabel('Гены трехпетельных белков', size=15)
ax.vlines([i * inds for i in range(1, 3)], *ax.get_ylim())
plt.tight_layout()
# plt.savefig('results/heatmap_all_atlas', facecolor='white', format='png')
plt.show()

In [ ]:
import os
print(os.listdir('atlas'))